In [1]:
import re
import os
import sys
import math
import time
import utils
import random
import layers
import dataset
import params
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from network import network

IndentationError: expected an indented block (network.py, line 128)

In [ ]:
#read params

with open("train.params") as f:
    learning_rate=float(params.read_param(f.readline()))
    batch_size=int(params.read_param(f.readline()))
    epoch=int(params.read_param(f.readline()))
   
    print("training with a learning rate of {} for {} epochs with batchs of size {}".format(learning_rate,epoch,batch_size))

In [ ]:
data_all=np.load("all_data.npy")  
label_all=np.load("all_labels.npy")
print(data_all.shape,label_all.shape)

#prep for training
net=network(learning_rate)
model_dir="model_{}_{}_{}".format(learning_rate,batch_size,epoch)
if not os.path.isdir(model_dir):
    os.makedirs(model_dir)

In [ ]:
# train network
loss_all=[]
acc_all=[]
N=len(data_all)
for e in range(epoch):
    start_time=time.time()
    perm=np.random.permutation(N)
    data_all=data_all[perm]
    label_all=label_all[perm]
    split_size=math.ceil(N/batch_size)
    data_batchs=np.array_split(data_all,split_size)
    label_batchs=np.array_split(label_all,split_size)
    loss_epoch=[]
    acc_epoch=[]

    batch_number=0
    for data,label in zip(data_batchs,label_batchs):
        batch_info=net.sess.run([net.mean_loss,net.mean_accuracy,net.optimizing_op],feed_dict={net.X:data, net.Y: label})
        loss_batch,acc_batch=batch_info[0],batch_info[1]
        loss_epoch.append(loss_batch)
        acc_epoch.append(acc_batch)
        batch_number+=1
        if batch_number%50==0:
            print("epoch_{:03d}-batch_{:03d}: loss={}, acc={}".format(e,batch_number,loss_batch,acc_batch))
    loss_all.append(loss_epoch)
    acc_all.append(acc_epoch)
    net.save("{}/epoch_{:03d}".format(model_dir,e),loss_all,acc_all)
    print("epoch %d took %d seconds"%(e,time.time()-start_time))

In [ ]:
np.load("/Users/micmelesse/Documents/thesis/model_1.0_10_3/epoch_002/losses.npy")